In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
import matplotlib.pyplot as plt

# Database setup
def setup_db():
    conn = sqlite3.connect("expenses.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS expenses (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            date TEXT,
            category TEXT,
            amount REAL
        )
    """)
    conn.commit()
    conn.close()

setup_db()

# Function to add expense
def add_expense():
    date = date_entry.get()
    category = category_var.get()
    amount = amount_entry.get()
    
    if not date or not category or not amount:
        messagebox.showerror("Error", "All fields are required!")
        return
    
    try:
        amount = float(amount)
        conn = sqlite3.connect("expenses.db")
        cursor = conn.cursor()
        cursor.execute("INSERT INTO expenses (date, category, amount) VALUES (?, ?, ?)", (date, category, amount))
        conn.commit()
        conn.close()
        messagebox.showinfo("Success", "Expense added successfully!")
        view_expenses()
    except ValueError:
        messagebox.showerror("Error", "Amount must be a number!")

# Function to view expenses
def view_expenses():
    filter_date = date_entry.get()
    filter_category = category_var.get()
    
    conn = sqlite3.connect("expenses.db")
    cursor = conn.cursor()
    query = "SELECT * FROM expenses"
    params = []
    
    if filter_date:
        query += " WHERE date = ?"
        params.append(filter_date)
    elif filter_category:
        query += " WHERE category = ?"
        params.append(filter_category)
    
    cursor.execute(query, params)
    records = cursor.fetchall()
    conn.close()
    
    for row in tree.get_children():
        tree.delete(row)
    
    for record in records:
        tree.insert("", "end", values=record)

# Function to delete expense
def delete_expense():
    selected_item = tree.selection()
    if not selected_item:
        messagebox.showerror("Error", "No expense selected!")
        return
    
    item = tree.item(selected_item)
    expense_id = item['values'][0]
    
    conn = sqlite3.connect("expenses.db")
    cursor = conn.cursor()
    cursor.execute("DELETE FROM expenses WHERE id = ?", (expense_id,))
    conn.commit()
    conn.close()
    
    messagebox.showinfo("Success", "Expense deleted successfully!")
    view_expenses()

# Function to visualize expenses
def open_visualization_window():
    visualization_window = tk.Toplevel(root)
    visualization_window.title("Expense Visualization")
    visualization_window.geometry("600x400")
    
    conn = sqlite3.connect("expenses.db")
    cursor = conn.cursor()
    cursor.execute("SELECT category, SUM(amount) FROM expenses GROUP BY category")
    data = cursor.fetchall()
    conn.close()
    
    if not data:
        messagebox.showerror("Error", "No data to visualize!")
        return
    
    categories, amounts = zip(*data)
    plt.figure(figsize=(6, 4))
    plt.pie(amounts, labels=categories, autopct="%.1f%%", startangle=140)
    plt.title("Expense Distribution")
    
    from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
    canvas = FigureCanvasTkAgg(plt.gcf(), master=visualization_window)
    canvas.draw()
    canvas.get_tk_widget().pack()

# GUI Setup
root = tk.Tk()
root.title("Expense Tracker")
root.geometry("600x500")

frame = tk.Frame(root)
frame.pack(pady=10)

tl_date = tk.Label(frame, text="Date (YYYY-MM-DD):")
tl_date.grid(row=0, column=0)
date_entry = tk.Entry(frame)
date_entry.grid(row=0, column=1)

category_var = tk.StringVar()
category_choices = ["Groceries", "Rent", "Entertainment", "Shopping", "Transportation", "Bills", "Other"]
category_var.set(category_choices[0])

tl_category = tk.Label(frame, text="Category:")
tl_category.grid(row=0, column=2)
category_dropdown = ttk.Combobox(frame, textvariable=category_var, values=category_choices)
category_dropdown.grid(row=0, column=3)

amount_label = tk.Label(frame, text="Amount:")
amount_label.grid(row=0, column=4)
amount_entry = tk.Entry(frame)
amount_entry.grid(row=0, column=5)

add_button = tk.Button(frame, text="Add Expense", command=add_expense, bg="green", fg="white")
add_button.grid(row=0, column=6)

view_button = tk.Button(root, text="View Expenses", command=view_expenses)
view_button.pack()

delete_button = tk.Button(root, text="Delete Selected Expense", command=delete_expense, bg="red", fg="white")
delete_button.pack()

visualize_button = tk.Button(root, text="Visualize Expenses", command=open_visualization_window)
visualize_button.pack()

columns = ("ID", "Date", "Category", "Amount")
tree = ttk.Treeview(root, columns=columns, show="headings")
for col in columns:
    tree.heading(col, text=col)
    tree.column(col, width=100)
tree.pack(expand=True, fill="both")

view_expenses()
root.mainloop()
